In [1]:
import os
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# Create your DataFrame
images = []
labels = []
label = 0
folder = r'C:\Users\Jason\Downloads\184a\AD-Detection\archive2\Combined Dataset\train'
for subfolder in os.listdir(folder):
    subfolder_path = os.path.join(folder, subfolder)
    for image_filename in os.listdir(subfolder_path):
        image_path = os.path.join(subfolder_path, image_filename)
        images.append(image_path)
        labels.append(label)
    label += 1

df = pd.DataFrame({'image': images, 'label': labels})
df

,image,label
0,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
1,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
2,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
3,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
4,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,0
...,...,...
11514,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3
11515,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3
11516,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3
11517,C:\Users\rohan\Desktop\184a\AD-Detection\archi...,3


In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], shuffle=True, random_state=42)

In [ ]:
class ADDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame containing image paths and labels.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Get image path and label
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]
        
        # Load image
        image = Image.open(img_path).convert("RGB")
        
        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label



# Define transformations
transform = transforms.Compose([
    transforms.Resize((176, 176)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Create Datasets for training and testing
train_dataset = ADDataset(dataframe=train_df, transform=transform)
test_dataset = ADDataset(dataframe=test_df, transform=transform)

print(len(train_dataset), len(test_dataset))


# Create DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Test the DataLoader
print("Training DataLoader:")
for images, labels in train_dataloader:
    print(images.shape)  # Should be [batch_size, 3, 176, 176]
    print(labels)
    break

print("Testing DataLoader:")
for images, labels in test_dataloader:
    print(images.shape)  # Should be [batch_size, 3, 176, 176]
    print(labels) 
    break

9215 2304
Training DataLoader:
torch.Size([16, 3, 176, 176])
tensor([2, 0, 1, 2, 3, 0, 1, 2, 1, 1, 1, 0, 1, 3, 1, 3])
Testing DataLoader:
torch.Size([16, 3, 176, 176])
tensor([1, 2, 0, 3, 2, 3, 2, 2, 0, 0, 1, 0, 2, 0, 1, 3])


In [7]:
class FeatureExtractionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FeatureExtractionBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.batch_norm(x)
        x = self.max_pool(x)
        return x


class ADCNN(nn.Module):
    def __init__(self, dropout_rate=0.25):
        super(ADCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Feature Extraction Blocks
        self.block1 = FeatureExtractionBlock(16, 32)  # Output: 32 channels
        self.block2 = FeatureExtractionBlock(32, 64)  # Output: 64 channels
        self.block3 = FeatureExtractionBlock(64, 128)  # Output: 128 channels
        self.block4 = FeatureExtractionBlock(128, 256)  # Output: 256 channels

        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.dropout2 = nn.Dropout(p=dropout_rate)
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 5 * 5, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64) 
        self.fc4 = nn.Linear(64, 4)
        
        # Softmax layer is applied in forward
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.dropout1(x)
        x = self.block4(x)
        x = self.dropout2(x)

        x = torch.flatten(x, start_dim=1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        #x = self.softmax(x)
        return x

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = ADCNN()

cuda:0


In [ ]:

for inputs, labels in train_dataloader:
    print(inputs.shape, labels.shape)
    break

def train_model(model, train_loader, num_epochs=50, learning_rate=0.001, device='cuda'):
    model.to(device)
    
    # Initialize RMSprop optimizer
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    
    # Loss function (CrossEntropy for classification)
    criterion = torch.nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0
        
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # Move data to the selected device
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calculate loss
            loss = criterion(outputs, labels)

            # Backward pass (backpropagation)
            loss.backward()

            # Update weights
            optimizer.step()

            # Track running loss
            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Print stats for the epoch
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

        if epoch % 10 == 0:
            torch.save(model.state_dict(), r'C:\Users\rohan\Desktop\184a\AD-Detection\trained_cnn2.pth')
     
    print('Training Finished!')


train_model(model, train_dataloader , num_epochs=50, learning_rate=0.001, device='cuda')
torch.save(model.state_dict(), r'C:\Users\rohan\Desktop\184a\AD-Detection\trained_cnn2.pth')

torch.Size([16, 3, 176, 176]) torch.Size([16])
Epoch [1/50], Loss: 3.3320, Accuracy: 47.21%
Epoch [2/50], Loss: 1.1902, Accuracy: 53.40%
Epoch [3/50], Loss: 1.3177, Accuracy: 56.78%
Epoch [4/50], Loss: 0.8394, Accuracy: 63.46%
Epoch [5/50], Loss: 0.6535, Accuracy: 70.27%
Epoch [6/50], Loss: 0.5680, Accuracy: 73.93%
Epoch [7/50], Loss: 0.5042, Accuracy: 77.65%
Epoch [8/50], Loss: 0.4364, Accuracy: 81.32%
Epoch [9/50], Loss: 0.3791, Accuracy: 83.72%
Epoch [10/50], Loss: 0.3172, Accuracy: 86.96%
Epoch [11/50], Loss: 0.2630, Accuracy: 89.07%
Epoch [12/50], Loss: 0.2166, Accuracy: 91.36%
Epoch [13/50], Loss: 0.1773, Accuracy: 93.09%
Epoch [14/50], Loss: 0.1497, Accuracy: 94.50%
Epoch [15/50], Loss: 0.1319, Accuracy: 95.18%
Epoch [16/50], Loss: 0.1185, Accuracy: 95.66%
Epoch [17/50], Loss: 0.1057, Accuracy: 96.15%
Epoch [18/50], Loss: 0.0824, Accuracy: 97.07%
Epoch [19/50], Loss: 0.0941, Accuracy: 96.91%
Epoch [20/50], Loss: 0.0738, Accuracy: 97.30%
Epoch [21/50], Loss: 0.0808, Accuracy: 97.

In [12]:
model = ADCNN().to(device)
model.load_state_dict(torch.load(r'C:\Users\rohan\Desktop\184a\AD-Detection\trained_cnn2.pth'))

C:\Users\rohan\AppData\Local\Temp\ipykernel_27624\81967438.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r'C:\Users\rohan\Desktop\184a

<All keys matched successfully>

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import itertools
import torch

def evaluate_model(model, test_loader, device='cuda'):
    y_pred_list = []
    y_target_list = []

    model.eval()
    with torch.no_grad():
        for inputs, labels in test_loader:
            # Move data to the selected device
            inputs, labels = inputs.to(device), labels.to(device)

            # Get model predictions
            outputs = model(inputs)
            _, y_pred = torch.max(outputs, 1)  # Predicted labels
            
            y_pred_list.append(y_pred.cpu().numpy())
            y_target_list.append(labels.cpu().numpy())

    # Flatten the predictions and targets
    y_pred_list = list(itertools.chain.from_iterable(y_pred_list))
    y_target_list = list(itertools.chain.from_iterable(y_target_list))

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_target_list, y_pred_list)
    print("Confusion Matrix of the Test Set")
    print("-----------")
    print(conf_matrix)

    # Calculate metrics
    precision = precision_score(y_target_list, y_pred_list, average='weighted')
    recall = recall_score(y_target_list, y_pred_list, average='weighted')
    f1 = f1_score(y_target_list, y_pred_list, average='weighted')

    print(f"Precision of the Model :\t{precision:.4f}")
    print(f"Recall of the Model    :\t{recall:.4f}")
    print(f"F1 Score of the Model  :\t{f1:.4f}")

evaluate_model(model, test_dataloader, device='cuda')


Confusion Matrix of the Test Set
-----------
[[537   2   6   3]
 [  0 514   0   0]
 [  2   0 630   8]
 [  6   0  12 584]]
Precision of the Model :	0.9831
Recall of the Model    :	0.9831
F1 Score of the Model  :	0.9831
